<h1>BERTの基本モデル理解

In [ ]:
! pip install transformers==4.5.0

#fugashi Mecab(形態素解析)のPythonラッパー
#unidic-lite 軽量の日本語辞書　Colabではおすすめ

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 700.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.0/698.0 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 30.7 MB/s eta 0:00:00
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658817 sha256=e466df62cfe1150f8f23cb1fd6036f282f6f7cd858fd322c96f47a72883d2f8e
  Stored in directory: /root/.cache/pip/wheels/b7/fd/e9/ea4459b868e6d2902e8d80e82dbacb6203e05b3b3a58c64966
  error: subprocess-exited-with-error
  
  × Building wheel for tokenize

In [ ]:
!pip install fugashi unidic-lite
import torch
from transformers import BertJapaneseTokenizer, BertModel
#東北大学の日本語モデルを使用
model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
#トークナイザ（文章をトークンに分解し、BERTに入力できる形に変換）を作成。学習済のトークナイザを使用可能
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

  Using cached fugashi-1.4.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.9 kB)
  Using cached unidic_lite-1.0.8-py3-none-any.whl
Using cached fugashi-1.4.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (698 kB)


In [ ]:
#トークン化（単語またはサブワード単位に分割）
#まず形態素解析で文を単語や品詞単位にわける。そして、WordPieceで未知の単語はサブワード単位に分割する
print(tokenizer.tokenize("明日は自然言語処理処理の勉強をしよう。"))
print(tokenizer.tokenize("明日はマシンラーニングの勉強をしよう。"))

['明日', 'は', '自然', '言語', '処理', '処理', 'の', '勉強', 'を', 'しよ', '##う', '。']
['明日', 'は', 'マシン', 'ラー', '##ニング', 'の', '勉強', 'を', 'しよ', '##う', '。']


In [ ]:
#エンコード（文章をトークン化後、それぞれのトークンをＩＤに変換する処理）
#IDは事前学習モデルが単語それぞれに一意のIDを割り当てている
input_ids = tokenizer.encode("明日は自然言語処理の勉強をしよう。")
print(input_ids)

[2, 11475, 9, 1757, 1882, 2762, 5, 8192, 11, 2132, 28489, 8, 3]


In [ ]:
#BERTでの学習では、トークン列の長さ（系列長）を同じにそろえる必要がある
#もし、系列長が揃える長さよりも短ければ、トークン列の末尾に特殊トークン[PAD]を足す。
#系列長が長ければ、必要な数だけ末尾のトークンを取り除く
#また、[PAD]はアテンションに関係ないので、どの部分にアテンションをかけるかを示す attention_maskも用意しておく

text = "明日の天気も晴れだ。"
#引数max_lengthは、特殊トークンを含めた最終的なID列の長さ
#引数padding="max_length"は、12トークンより小さいければ[PADトークンで埋める]
#引数truncation=Trueは、長すぎるテキストは後ろを切って12トークン以内となる
#切り捨てによって、文の重要な情報が消える可能性あり
encoding = tokenizer(text, max_length=12, padding="max_length", truncation=True)

print("# Encoding")
print(encoding)

token = tokenizer.convert_ids_to_tokens(encoding["input_ids"])
print(" tokens:")
print(token)

# Encoding
{'input_ids': [2, 11475, 5, 11385, 28, 16577, 75, 8, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}
 tokens:
['[CLS]', '明日', 'の', '天気', 'も', '晴れ', 'だ', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]']


In [ ]:
#リスト化した文章にも実施可能
text_list = ["明日の天気も晴れだ。","パソコンが急に動かなくなった。"]
tokenizer(text_list, max_length=10, padding="max_length", truncation=True)

{'input_ids': [[2, 11475, 5, 11385, 28, 16577, 75, 8, 3, 0], [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
#それぞれのトークンに対するベクトルを出力
#個別の言語タスクに特化したモデルが適用されているか動作確認のため、ベーシックなBERTmodelで説明
bert = BertModel.from_pretrained(model_name)

bert.config
#モデルのパラメータ数は約１億１千万


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

BertConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

In [ ]:
text_list = ["明日は自然言語処理処理の勉強をしよう。","明日はマシンラーニングの勉強をしよう。"]

#引数return_tensors="pt"でテンソルの形に整形
encoding = tokenizer(text_list, max_length=32, padding="max_length",truncation=True, return_tensors="pt")

#トークン列が[PAD]を含まない場合は、input_idsのみを入力しても同じ出力となる
output = bert(
    input_ids=encoding["input_ids"],
    attention_mask=encoding["attention_mask"],
    token_type_ids=encoding["token_type_ids"]
)

last_hidden_state = output.last_hidden_state

In [ ]:
#この３次元配列は（バッチサイズ（まとめて処理する文章の数）、系列長、隠れ状態の次元）
print(last_hidden_state.size())

torch.Size([2, 32, 768])
